In [1]:
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv("https://storeacc1001.blob.core.windows.net/data/fatigue/fatigue_data_13_sets.csv")
df.head()

,FILE01,FILE02,FILE03,FILE04,FILE05,FILE06,FILE07,FILE08,FILE09,FILE10,FILE11,FILE12,FILE13
0,Narrow band.,White noise,High pass,Ramp up,Ramp down,Three-peaked,4x3-peak,4x3-peak atten,Three spikes,Ten spikes,High spike,Field A,Field B
1,0.9997,0.745,0.935,0.866,0.645,0.673,0.732,0.712,0.592,0.744,0.997,0.615,0.791
2,0,1,0.01,0.00099975,1.99900025,0.00999833,0.000157,0.000285455,0.023853142,0.007156083,0.00991061,0.026344231,0.000444716
3,0,1,0.01,0.00099975,1.99900025,0.00999833,0.000157,0.000285455,0.023853142,0.007156083,0.00991061,0.025870035,0.002221359
4,0,1,0.01,0.001499625,1.998500375,0.00999833,0.000157,0.000285455,0.023853142,0.007156083,0.00991061,0.025395839,0.003998001


In [2]:
out = widgets.Output(layout={'border': '8px solid blue'})
display(out)

Output(layout=Layout(border_bottom='8px solid blue', border_left='8px solid blue', border_right='8px solid blu…

In [3]:
out2 = widgets.Output(layout={'border': '1px solid black'})
display(out2)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

SCALE TO AN RMS OF 1 IF A NEW FILE IS BEING READ:

If FILE = 26 Then

GSUM = 0

For I = 1 To 4000

GSUM = GSUM + GOMEGA(I)

Next I

For I = 1 To 4000

GOMEGA(I) = GOMEGA(I) * 4000 / GSUM

Next I

End If

In [4]:
def newfile_handler():
    gsum = 0
    for i in range(4000):
        gsum += gomega(i)
    #sum command available
    array = np.array(gomega)
    array = array * (4000 / gsum)
    gomega = array.tolist()
    return()

In [5]:
file_opt = list(df[1:1])
#file_opt.append("NEW FILE")
file_name = "FILE01"
    
def dropdown_eventhandler(change):
    global file_name
    file_name = change.new

    
def psd_moments(gintervals, gomega):
    stripwidth = 1 / gintervals
    m0 = 0
    m1= 0
    m2 = 0
    m4 = 0
    for i in range(gintervals):
        m0 += gomega[i] * stripwidth
        m1 += gomega[i] * stripwidth * ((i - 0.5) * stripwidth)
        m2 += gomega[i] * stripwidth * ((i - 0.5) * stripwidth)**2
        m4 += gomega[i] * stripwidth * ((i - 0.5) * stripwidth)**4
    sigma = m0**.5
    xm = m1 / m0 * (m2 / m4)**.5
    gamma = (m2**2 / (m0 * m4))**.5
    maxrange = 10 * sigma
    return sigma, xm, gamma, maxrange


def rrd(sigma, xm, gamma, maxrange, probints):
    prr = [0] * probints
    d1 = 2 * (xm - gamma**2) / (1 + gamma**2)
    r = (gamma - xm - d1**2) / (1 - gamma -d1 + d1**2)
    d2 = (1 - gamma - d1 + d1**2) / (1 - r)
    d3 = 1 - d1 - d2
    q = 1.25 * (gamma - d3 - (d2 * r)) / d1
    stripwidth = maxrange / (probints - 1)
    for j in range(probints):
        x = stripwidth * (j + 0.5)
        z = x / (2 * sigma)
        xrr = (-z) / q
        yrr = (-z) * z / (2 * r * r)
        zrr = (-z) * z / 2
        prr[j] = (d1 / q * math.exp(xrr) + d2 * z / (r * r) * math.exp(yrr) + d3 * z * math.exp(zrr)) / (2 * sigma)
    return prr
    
def prr_rescale(prr, gamma):
    prsum = 0
    for i in range(1000):
        prsum += prr[i]
    for i in range(1000):
        prr[i] = prr[i]/ prsum
    
    #for i in range(1000):
        #prr[i] = prr[i] * (0.3 * gamma + 0.7)
    
    return prr


def fatiguelife_estimate(rms, bee, aee, flimit, prr, freqmax):
    dam = [0] * 1000
    for i in range(1000):
        dam[i] = (prr[i] / aee) * ((i + 0.5) * rms / 100)**bee
        
    x = int(100 * 2 * flimit) + 1
    damsum = 0
    for i in range(1000):
        damsum += dam[i]
    return 1 / (damsum * freqmax * 3600)


def experiment(b):    
    gintervals = 4000
    probints = 1000
    rms = 100
    bee = 5
    aee = 3.0 * 10**21
    flimit = 0
    freqmax = 100
    intervals = []
    frequ_1 = []
    
    with out:
        clear_output()
    with out2:
        clear_output()
    
    for i in range(probints):
        intervals.append(i / 1000)
        
    if file_name == "NEW FILE":
        gomega = newfile_handler()
    else:
        df = pd.read_csv("https://storeacc1001.blob.core.windows.net/data/fatigue/fatigue_data_13_sets.csv")
        print(file_name)
        gomega = list((df[file_name])[2::].astype(float))
        #print(gomega)
        
    for i in range(1, gintervals + 1):
        frequ_1.append((i - 0.5) / gintervals)
        
    sigma, xm, gamma, maxrange = psd_moments(gintervals, gomega)
    prr = rrd(sigma, xm, gamma, maxrange, probints)
    prr = prr_rescale(prr, gamma)
    print(fatiguelife_estimate(rms, bee, aee, flimit, prr, freqmax))
    #print(prr)
    #x = np.array(intervals)
    #y = np.array(prr)
    plt.plot(intervals, prr)
    plt.title("Rainflow range distribution")
    plt.xlabel("Interval, I")
    plt.ylabel("PRR[I]")
    plt.ylim(ymin = 0)
    plt.xlim(xmin = 0)
    with out:
        plt.show()

    plt.plot(frequ_1, gomega)
    plt.title("PSD of chosen file")
    plt.xlabel("Frequency")
    plt.ylabel("G(w)")
    plt.ylim(ymin = 0)
    plt.xlim(xmin = 0)
    with out2:  
        plt.show()


        
drop_down = widgets.Dropdown(
    options=file_opt,
    value='FILE01',
    description='FILE:',
    disabled=False,
)
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

drop_down.observe(dropdown_eventhandler, names='value')
display(drop_down)
display(button)
button.on_click(experiment)
#x = list(df["FILE04"])[2::]


Dropdown(description='FILE:', options=('FILE01', 'FILE02', 'FILE03', 'FILE04', 'FILE05', 'FILE06', 'FILE07', '…

Button(button_style='info', description='Click me', style=ButtonStyle(), tooltip='Click me')

FILE01
1393.2441644289331


Sub fatigue()

'***********************************************************************

'                                                                      *

' This program calculates the probability density distribution of

' rainflow ranges from any given PSD, using the Dirlik expression.

' It then estimates a fatigue life under that PSD, given data for the

' S/N curve and the highest frequency present in the PSD

'                                                                      *

'                 Frank Sherratt October 2001                          *

'                                                                      *

'***********************************************************************

      Dim GOMEGA(4001), FREQU1(4001), PRR(1001), NBUNIT(1001) As Double

      Dim DAM(1001) As Double

'**********************************************************************

' READ IN OR SET BASIC DATA

'**********************************************************************

      GINTERVALS = 4000 'number of intervals on the PSD

      PROBINTS = 1000 ' number of intervals in the rainflow distribution

      FILE = Cells(4, 4) ' Read in the file number

      FILE = FILE + 26 ' set the column for that file

' read in RMS and S/N data

      RMS = Cells(5, 4)

      BEE = Cells(6, 4)

      AEE = Cells(7, 4)

      FLIMIT = Cells(8, 4)

      FREQMAX = Cells(9, 4)

'***********************************************************************

' THIS SECTION READS THE PSD ARRAY FROM DATA IN THE NOMINATED COLUMN,  *

' SETTING UP "GINTERVALS" OF WIDTH "D" ON THE FREQUENCY AXIS           *

' IF 'FILE=0 IT READS 4000 VALUES FROM COLUMN Z AND NORMALISES THEM   *

' TO UNIT RMS BEFORE CONTINUING.                                       *

'***********************************************************************

      For I = 1 To GINTERVALS

      FREQU1(I) = (I - 0.5) / GINTERVALS 'FREQU1 IS THE X CO-ORDINATE OF THE PSD

      GOMEGA(I) = Cells(I, FILE)

      Next I

'***********************************************************************

' SCALE TO AN RMS OF 1 IF A NEW FILE IS BEING READ                     *

'***********************************************************************

      If FILE = 26 Then

      GSUM = 0

      For I = 1 To 4000

      GSUM = GSUM + GOMEGA(I)

      Next I

      For I = 1 To 4000

      GOMEGA(I) = GOMEGA(I) * 4000 / GSUM

      Next I

      End If

'***********************************************************************

' LIST G(w) VALUES TO Col. L FOR USE IN DISPLAY                        *

'***********************************************************************

      For I = 1 To 1000

      Cells(I + 2, 12).Value = GOMEGA(I * 4)

      Next I

'***********************************************************************

' START CALCULATION OF PRR

'***********************************************************************

'***********************************************************************

' CALCULATE THE REQUIRED MOMENTS OF THE PSD

'***********************************************************************

      STRIPWIDTH = 1 / GINTERVALS

      M0 = 0

      M1 = 0

      M2 = 0

      M4 = 0 ' initialise M values

      For I = 1 To GINTERVALS

      M0 = M0 + GOMEGA(I) * STRIPWIDTH

      M1 = M1 + GOMEGA(I) * STRIPWIDTH * ((I - 0.5) * STRIPWIDTH)

      M2 = M2 + GOMEGA(I) * STRIPWIDTH * ((I - 0.5) * STRIPWIDTH) ^ 2

      M4 = M4 + GOMEGA(I) * STRIPWIDTH * ((I - 0.5) * STRIPWIDTH) ^ 4

      Next I

      SIGMA = Sqr(M0)

      XM = M1 / M0 * Sqr(M2 / M4)

      GAMMA = Sqr(M2 ^ 2 / (M0 * M4))

      MAXRANGE = 10 * SIGMA

'***********************************************************************

' START OF SECTION TO PRODUCE RAINFLOW RANGE DISTRIBUTION              *

'***********************************************************************

      D1 = 2 * (XM - GAMMA ^ 2) / (1 + GAMMA ^ 2)

      R = (GAMMA - XM - D1 ^ 2) / (1 - GAMMA - D1 + D1 ^ 2)

      D2 = (1 - GAMMA - D1 + D1 ^ 2) / (1 - R)

      D3 = 1 - D1 - D2

      Q = 1.25 * (GAMMA - D3 - (D2 * R)) / D1

      STRIPWIDTH = MAXRANGE / (PROBINTS - 1)

      For J = 1 To PROBINTS

      X = STRIPWIDTH * (J - 0.5)

      Z = X / (2 * SIGMA)

      XRR = (-Z) / Q

      YRR = (-Z) * Z / (2 * R * R)

      ZRR = (-Z) * Z / 2

      PRR(J) = (D1 / Q * Exp(XRR) + D2 * Z / (R * R) * Exp(YRR) + D3 * Z * Exp(ZRR)) / (2 * SIGMA)

      Next J

'************************************************************************

' re-scale PRR to unit probability

      PRSUM = 0

      For I = 1 To 1000

      PRSUM = PRSUM + PRR(I)

      Next I

      For I = 1 To 1000

      PRR(I) = PRR(I) / PRSUM

      Next I

' output modified PRR to column J

      For I = 1 To 1000

      Cells(I + 2, 10) = PRR(I)

      Next I

'*************************************************************************

' Carry out a fatigue life estimate

'*************************************************************************

      For I = 1 To 1000

      DAM(I) = (PRR(I) / AEE) * ((I - 0.5) * RMS / 100) ^ BEE

      Next I

'determine I equivalent of the fatigue limit

      X = Int(100 * 2 * FLIMIT) + 1

      DAMSUM = 0

      For I = X To 1000

      DAMSUM = DAMSUM + DAM(I)

      Next I

      Cells(10, 4) = 1 / (DAMSUM * FREQMAX * 3600)

 

2000  End Sub

 

'***********************************************************************

 

Sub CONTINUE()

'Continue the program

End Sub